# 점수 산출 방식
1. 데이터 입력
2. 5개 모델을 사용해 각각 결과 예측 후 평균값 산출
3. 1에 가까울수록 정상기사

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import random
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
tqdm.pandas()
from transformers import AutoTokenizer

In [ ]:
root_dir = "/content/drive/MyDrive/"
project_folder = "project"
os.chdir(os.path.join(root_dir,project_folder))

In [ ]:
import easydict

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"current device : {device}")

args = easydict.EasyDict({
        "seed":42,
        "seq_max_len":128,
        "batch_size": 32,
        "n_splits" : 5,
        "num_workers":2,
        "dp": 0.0,
        "model": "kobert" 
    })

current device : cpu


In [ ]:
def seed_everything(seed: int = 42, contain_cuda: bool = False):
    os.environ["PYTHONHASHSEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    print(f"Seed set as {seed}")

seed_everything(args.seed)

Seed set as 42


In [ ]:
def tokenized_dataset(args, dataset, tokenizer):
    lst_title = dataset["title"].tolist()
    lst_content = dataset["content"].tolist()

    tokenized_sentences = tokenizer(
        lst_title,
        lst_content,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=args.seq_max_len,
        add_special_tokens=True
    )

    return tokenized_sentences

In [ ]:
class NewsDataset(Dataset):
    def __init__(self, tokenized_dataset, label):
        self.tokenized_dataset = tokenized_dataset
        self.label = label

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.tokenized_dataset.items()}
        item["label"] = torch.tensor(self.label[idx])
        return item

    def __len__(self):
        return len(self.label)

## Prediction

In [ ]:
class kobert_Classifier(nn.Module):
    def __init__(self, bert, hidden_size=768, num_classes=2, dr_rate=0.0):
        super(kobert_Classifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.pooler = nn.Linear(hidden_size, hidden_size)
        self.classifier = nn.Linear(hidden_size, num_classes)
        torch.nn.init.xavier_uniform_(self.classifier.weight)
        self.dropout = nn.Dropout(p=dr_rate)

    def forward(self, token_ids, attention_mask, segment_ids):
        out = self.bert(input_ids=token_ids, attention_mask=attention_mask, token_type_ids=segment_ids)[0]
        out = out[:, 0, :]
        out = self.pooler(out)
        out = torch.nn.functional.tanh(out)

        if self.dr_rate:
            out = self.dropout(out)
        
        return self.classifier(out)

In [ ]:
def get_tokenizer(args):
    if args.model == "kobert":
        tokenizer = AutoTokenizer.from_pretrained("kykim/bert-kor-base")

    return tokenizer

In [ ]:
def load_test_dataset(args, title, content, tokenizer):
  title = title
  content = content
  test_dataset = pd.DataFrame({"title" : [title], "content" : [content], "label" : [0]})
  test_label = test_dataset["label"].values

  tokenized_test = tokenized_dataset(args, test_dataset, tokenizer)
  return tokenized_test, test_label

def test_ensemble_main(args, title, content, ensemble="soft"):
    model = torch.load("./models/model.pt", map_location=device)
    tokenizer = get_tokenizer(args)
    # load test datset
    test_dataset, test_label = load_test_dataset(args, title, content, tokenizer)
    test_dataset = NewsDataset(test_dataset, test_label)
    testloader = DataLoader(test_dataset,
                    shuffle=False,
                    batch_size=args.batch_size,
                    num_workers=args.num_workers,
                    )
    
    if ensemble == "soft":
        all_fold_p = np.zeros((1, 2))  
        for idx in tqdm(range(1, args.n_splits+1)):
            load_path = f"./models/{idx}-fold/best.pt"
            model.load_state_dict(torch.load(load_path,map_location=torch.device("cpu")))
            model.to(device)
            model.eval()

            progress_bar = tqdm(enumerate(testloader), total=len(testloader), leave=True, position=0,)
            for i, data in progress_bar:
                with torch.no_grad():
                    logits = model(
                        data["input_ids"].to(device),
                        data["attention_mask"].to(device),
                        data["token_type_ids"].to(device)
                    )
                if i==0:
                    one_fold_logits = logits
                    p = torch.nn.functional.softmax(one_fold_logits, dim=1)

                else:
                    one_fold_logits = torch.cat([one_fold_logits,logits],dim=0)
                    p = torch.nn.functional.softmax(one_fold_logits, dim=0)
            
            p = p.squeeze(0).detach().cpu().numpy()

            all_fold_p += p
        all_fold_p = all_fold_p / 5
        score = all_fold_p[0][1]
        return all_fold_p, score

In [ ]:
title = '1억 7천짜리 신형 BMW 올라타 방방 뛰며 앞유리 박살 낸 11살 초등학생 빌런'
content = '한 여자 초등학생이 아파트 지하 주차장에 세워진  위에 올라타 크게 훼손하는 영상이 공개됐다.그런데 이 초등학생의 부모가 피해자의 배상 요구를 막무가내로 거절해 논란이 일고 있다.지난 21일현지 시간 중국 매체 소후는 중국 장쑤성 난징시에서 한 여자 초등학생이 아파트 지하 주차장에 세워진 에 칼로 그림을 그리고 차에 올라타 전면 유리창을 밟아 깨뜨리는 영상을 공개했다.초등학생 양이 파손한 는 중국 현지에서 시가 100만 위안한화 약 1억 7000만 원에 판매되는 4 모델인 것으로 알려졌다. 차주는 출장 후 집에 돌아오니 차량 전면 유리가 부서지고 칼로 고의로 그은 흔적을 발견한 후 아파트 관리사무소를 찾아  영상을 재생해 범인인 양을 찾은 것으로 알려졌다. 영상에는 양이 킥보드를 타고 주차장을 이동하던 중 노란색 를 발견한 뒤 차량에 다가오는 장면이 그대로 담겨있었다.평소 좀처럼 구경하기 힘든 노란색 차량을 발견한 양은 호기심에 자동차에 다가가 황당하게도 훼손을 시작했다. 양은 소지했던 칼과 볼펜 등으로 차량에 그림을 그리기 시작했다. 양이 칼로 그으며 그림을 그리면서  차량 전면은 칼 자국이 심하게 팬 상태가 됐다.심지어 양은 자동차 위에 올라가 두 주먹으로 유리창을 치는 동작을 반복했는데 이 행동으로 의 유리창이 모두 깨지고 말았다.차주 씨는 당시 양의 부모에게 연락을 취했으나 그들은 자신의 딸이 차량 전면 유리를 깼다는 주장은 허무맹랑한 거짓말이라고 오히려 큰 소리를 쳤다라며 당황스러웠던 당시를 설명했다.양의 부모는 씨에게 11세의 소녀가 어떻게 차량 유리를 훼손할 만큼 힘이 있겠느냐라고 오히려 반문했다.씨는  영상을 근거로 양의 부모에게 배상을 논의했다. 하지만 양의 부모는 너도 나중에 아이를 낳을 것이 아니냐면서 그때 너도 네 아이를 데리고 와서 내 차 유리를 깨면 된다. 배상할 수 없다는 반응을 보였다고 한다.양의 경우 14세 미만 어린이로 처벌이 어려워 현재 자오씨는 그의 부모에게 수 천만 원 상당의 비용을 청구하는 소송을 제기할 예정으로 알려졌다.저작권자  무단전재 및 재배포 금지국민들 무관심에도 묵묵히 훈련해 도쿄올림픽 금메달 도전하는 한국 국가대표 선수들국내에서 쉽게 접하기 어려운 스포츠로 비인기 종목에 속하지만 메달에 도전하는 선수들을 모아봤다....연인처럼 진한 스킨십하며 커플 화보 찍은 솔라양치승 관장 사진남성 매거진 맨즈헬스 코리아가 솔라와 양치승이 참여한 화보 사진을 공개했다....한국 이어 미국 선수단도 도쿄 올림픽 후쿠시마산 음식 손절했다미국이 도쿄 올림픽 기간 동안 32에 달하는 음식을 자체 조달하기로 했다....'

In [ ]:
method = "soft"
all_fold_p, score = test_ensemble_main(args, title, content, ensemble=method)

  0%|          | 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1949: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
100%|██████████| 5/5 [00:19<00:00,  3.89s/it]


In [ ]:
score

0.09922203272581101